In [6]:
import easyocr
import os.path
import numpy as np

In [7]:
reader = easyocr.Reader(['pt'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [39]:
def detection_matrix(list : list) -> list:
    matrix = []
    for j, element in enumerate(list):
        cord, detc, _ = element
        letters = [letter.upper() for letter in detc if letter != ' ']
        letters = ''.join(letters).replace("0", "O")
        line = []
        rigth = int(cord[1][0])
        left = int(cord[0][0])
        upper = int(cord[0][1])
        lower_begin = int(cord[3][1])
        lower_end = int(cord[2][1])
        width = round((left - rigth) / len(letters))
        height_variation = round((lower_end - lower_begin) / len(letters)) #se for positivo ent desceu
        for i in range(len(letters)):
            left_letter = left + (width * i)
            rigth_letter = rigth + (width * i)
            upper_letter = upper + (height_variation * i)
            lower_letter = lower_begin + (height_variation * i)
            cord_letter = [left_letter, upper_letter, rigth_letter, lower_letter]
            line.append((cord_letter, letters[i]))
        matrix.append(line)
    return matrix



def search_direction(matrix, r : int, c : int, word : str, direction: str):
    # return -> index of last letter (role, colum)
    # r -> role
    # c -> column 
    # dr -> direction role
    # dc -> direction column
    match direction:
        case 'search_ritgh':
            dr = 0
            dc = 1
        case 'search_left':
            dr = 0
            dc = -1
        case 'search_down':
            dr = 1
            dc = 0
        case 'search_up':
            dr = -1
            dc = 0
        case 'search_up_rigth':
            dr = -1
            dc = 1
        case 'search_up_left':
            dr = -1
            dc = -1
        case 'search_down_rigth':
            dr = 1
            dc = 1
        case 'search_down_left':
            dr = 1
            dc = -1
    try:
        for i, letter in enumerate(word):
            if (matrix[r+(i * dr)][c + (i* dc)]).lower() == letter.lower():
                if i == len(word) - 1:
                    last_index = (r+(i * dr), c + (i* dc)) 
                continue
            else:
                return None 
        return last_index
    except (IndexError):
        return None

def words_matrix_position(matrix : list, words_list : list) -> list:
    directions = ['search_ritgh', 'search_left', 'search_down', 'search_up', 'search_up_rigth', 'search_up_left', 'search_down_rigth', 'search_down_left']
    position_pairs = []  #list of tuples (r1, c1, r2, c2)
    for r in range(len(matrix)):
        for c in range(len(matrix[r])):
                for word in words_list:
                    # checks if the words starts whit the matrix letter
                    if matrix[r][c].lower() == word[0].lower():
                        # try all the directions until one of them works, break if works and try other word
                        for direction in directions:
                            last_index = search_direction(matrix, r, c, word, direction)
                            if last_index is not None:
                                position_pairs.append((r, c , last_index[0], last_index[1]))
                                words_list.remove(word)
                                break 
    return position_pairs

In [9]:
if os.path.exists('.\data\wordsearch.png'):
    result = reader.readtext('.\data\wordsearch.png')
else:
    print('fail')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\pedro\AppData\Local\Temp\ipykernel_23140\3460886727.py:1: SyntaxWarning: invalid escape sequence '\d'
  if os.path.exists('.\data\wordsearch.png'):
C:\Users\pedro\AppData\Local\Temp\ipykernel_23140\3460886727.py:2: SyntaxWarning: invalid escape sequence '\d'
  result = reader.readtext('.\data\wordsearch.png')
d:\pedro\Programacao\.Projetos\Python\ocr-wordsearch-solver-EasyOcr\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [50]:
words_list = [word for  _, word, _ in result if " " not in word]
search_word_list = [res for res in result if " " in res[1]]
matrix = detection_matrix(search_word_list)
letter_matrix = [[letter for _, letter in line] for line in matrix]
positions_pairs = words_matrix_position(letter_matrix, words_list)
print(positions_pairs)
if words_list:
    print("Não foi possivel detectar a/as seguintes palavras:")
    for word in words_list:
        print(word)

[(0, 2, 10, 12), (0, 4, 8, 4), (0, 9, 0, 21), (1, 7, 6, 12), (1, 9, 7, 15), (1, 18, 1, 8), (2, 9, 7, 9), (2, 21, 2, 15), (3, 3, 7, 7), (4, 11, 11, 18), (4, 16, 1, 19), (5, 14, 5, 18), (6, 0, 12, 6), (6, 13, 6, 21), (7, 2, 7, 0), (7, 6, 0, 6), (8, 9, 6, 7), (8, 13, 3, 13), (9, 4, 9, 11), (9, 12, 4, 7), (11, 9, 11, 15), (11, 19, 4, 19), (13, 16, 13, 19), (14, 8, 14, 13), (14, 16, 14, 21)]
Não foi possivel detectar a/as seguintes palavras:
0
ADICIONAL
MODO
ROUBARAM
GARANTIA
CRÉDITO
NUBANK
SEGURO
DIRETO
JUNTAS
BLOQUEAR
VALOR
ESCURO
VIDAS
CANAL
LIMITES
